# 머신러닝   
       
12주 1강: 나이브 베이즈 분류기

이번 시간에는 베이즈 이론을 이용한 나이브 베이즈 분류기를 구현해봅시다 

## 간단한 베이즈 분류기 구현하기
* 교과서 예제인 베이즈 분류기를 구현해 봅시다.

### 데이터 만들기
* 일단 교과서의 표 11-1을 참고해 봅시다. 표 11-1의 데이터를 DataFrame으로 만들면 아래와 같습니다

In [ ]:
import pandas as pd
import numpy as np

viagra_spam = {'viagra': [1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1],
               'spam': [1,0,0,0,0,0,1,0,1,0, 0,0,0,0,0,0,0,1,1,1]}
# 교과서는 df로 되어있으나, 겹치는 이름을 안 만들기 위해...
df_spam = pd.DataFrame(viagra_spam, columns = ['viagra', 'spam']) 
np_data = df_spam.values 

In [ ]:
#df_spam

### 확률값들을 구해봅시다


* 메일에 viagra가 있을 확률: $P(viagra)$

In [ ]:
p_viagra = sum(np_data[:, 0] == 1) / len(np_data)
p_viagra

* 메일이 spam일 확률: $P(spam)$

In [ ]:
p_spam = sum(np_data[:, 1] == 1) / len(np_data)
p_spam

* Viagra를 포함하면서 스팸메일일 확률: $P(viagra \cap spam)$

In [ ]:
p_v_cap_s = sum((np_data[:, 0] == 1) & (np_data[:, 1] == 1)) / len(np_data)
p_v_cap_s

* Viagra를 포함하지 않으면서 스팸메일일 확률: $P(\neg viagra \cap spam)$

In [ ]:
p_n_v_cap_s = sum((np_data[:, 0] == 0) & (np_data[:, 1] == 1)) / len(np_data)
p_n_v_cap_s

* 위 확률들을 정리해서 $P(spam|viagra)$ 를 구해봅시다

$P(spam|viagra) = \frac{P(viagra|spam)*P(spam)}{P(viagra)} = \frac{P(viagra \cap spam) / P(spam) * P(spam)}{P(viagra)}$

In [ ]:
# 사실 수식적으론 두개가 동일하죠. 하지만 우리는 원칙상으로 교집합의 확률을 "모른다" 라고 일단 가정을 해 봅시다

# p_spam * (p_v_cap_s / p_spam ) / p_viagra
p_v_cap_s / p_viagra 

* 이제 비아그라가 없는데 스팸메일일 확률을 구해봅시다 

$P(spam|\neg viagra) = \frac{P(\neg viagra|spam)*P(spam)}{P(\neg viagra)} = \frac{P(\neg viagra \cap spam) / P(spam) * P(spam)}{P(\neg viagra)}$

In [ ]:
p_spam * (p_n_v_cap_s / p_spam ) / (1-p_viagra)

### 나이브 베이지안 분류기 만들기
* 간단한 베이즈 분류기는 하나의 변수만 고려합니다.
* 하지만 실제로는 다양한 변수가 작용할 수 있습니다.
  * 예를 들어 viagra 말고, 다른 약의 이름이 있다던지....
* 교과서 예제인 fraud 데이터를 사용해 봅시다. 참고로 fraud는 "사기" 라는 뜻입니다. 
  * 이 데이터는 3개의 feature 를 사용해서 대출사기 여부를 판단하는 데이터입니다. 

In [ ]:
import pandas as pd
import numpy as np

data_url = "W12/fraud.csv"
df_fraud = pd.read_csv(data_url, sep=',')
df_fraud.head()

* 일단 T/F 데이터를 one-hot-encoding으로 처리해 봅시다

In [ ]:
del df_fraud["ID"]
Y_data = df_fraud.pop("Fraud")
Y_data = Y_data.values
x_df = pd.get_dummies(df_fraud)
x_df.head(10).T


#arrears 는 연체를 뜻합니다.

* 모델 입력을 위해 넘파이 배열 형태로 X 데이터를 바꿔줍시다. 

In [ ]:
x_data = x_df.values
x_data

* 나이브 베이지안은 독립 사건을 가정합니다. 그래서 아래와 같은 수식을 쓸 수 있습니다.

$P(Y_c|X_1, \cdots, X_n)=P(Y_c)\prod_{i=1}^{n}\frac{P(X_i|Y_c)}{P(X_i)}=\frac{P(Y_c)\prod_{i=1}^{n}P(X_i|Y_c)}{\prod_{i=1}^{n}P(X_i)}$

where $Y_c$ is a label

* 먼저 P(Y_c = True)와 P(Y_c = False)를 구해보ㅂ니다

In [ ]:
P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True,P_Y_False

* 다음으로 $P(X_i | Y_c)$를 표현하기 위해 $Y_c$의 라벨을 기반으로 한 값을 `np.where`로 출력해 봅시다
  * 참조 https://numpy.org/doc/stable/reference/generated/numpy.where.html
* 이 경우에는 True의 index를 반환합니다

In [ ]:
np.where(Y_data)

* True와 False의 위치를 반환받습니다

In [ ]:
ix_Y_True = np.where(Y_data)
ix_Y_False = np.where(Y_data==False)

ix_Y_True, ix_Y_False

In [ ]:
x_data.sum(axis=0)/len(Y_data)

* $P(X_i|Y_{True})$ 을 구하려면 아래처럼 하면 됩니다. 
  * x의 각 feature에 대해서, Y_data가 True인 케이스 중 몇 개나 그 Feature가 True였는지를 계산합니다. 
* 덤으로 $P(X_i|Y_{False})$도 구해봅시다. 

In [ ]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0)) / sum(Y_data==True)
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)) / sum(Y_data==False)

p_x_y_true, p_x_y_false

* 이제 가상의 Test Data를 봅시다.
  * `x_test = [0,1,0,0,0,1,0,0,1,0]`

* 교과서는 아래처럼 원래 수식이 아니라 이상한 형태로 구합니다.
  * 이 식은 실제 확률을 구해주지 않습니다

In [ ]:
print(x_df.columns)
x_test = [0,1,0,0,0,1,0,0,1,0]

p_y_true_test = P_Y_True + p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False + p_x_y_false.dot(x_test)

print(p_y_true_test , p_y_false_test)

In [ ]:
p_y_true_test < p_y_false_test
# 그래서, false일 확률이 더 높습니다

* 이렇게 구한 이유는 실제 확률 식에 위 값들을 대입하면 제대로 구해지지 않기 떄문입니다
  * x_test에 p_x_y_true 의 성분에 0 값이 있기 때문이죠. 

In [ ]:
p_y_true_test = P_Y_True * np.prod(p_x_y_true) / np.prod(x_data.sum(axis=0)/len(Y_data))
p_y_false_test = P_Y_False * np.prod(np.multiply(p_x_y_false, x_test)) / np.prod(x_data.sum(axis=0)/len(Y_data))

print(p_y_true_test , p_y_false_test)

## Scikit-learn을 이용한 나이브 베이지안 분류기
* 우리의 구현체에는 무언가 문제가 있긴 했습니다.
  * 그래도 전체 과정은 대충 알겠지요?
* 이제 "이미 잘 구현된" scikit-learn을 써 봅시다. 
* 먼저 문장을 sports/not sports로 구분하는 간단한 모델을 만들어봅시다.

In [ ]:
y_example_text = ["Sports", "Not sports", "Sports", "Sports", "Not sports"]
y_example = [1 if c=="Sports" else 0 for c in y_example_text ]
text_example = ["A great game game", 
                "The The election was over",
                "Very clean game match",
                "A clean but forgettable game game",
                "It was a close election", ]

* 이러한 텍스트 데이터에 대해서는 먼저 텍스트 데이터를 벡터화시켜야 합니다.  
  * 다양한 기법이 있지만, 여기서는 가장 단순한 Bag-of-words를 써보겠습니다.
  * BOW는 one-hot-encoding으로 단어를 인덱스화 시키고, 각 문장에 특정 단어가 몇 개나 들어있는지를 가지고 벡터화 시킵니다
  * `CountVectorizer`로 변환합시다 (이것 이외에도 다른 vectorizer들이 많긴 합니다)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

countvect_example = CountVectorizer()
X_example = countvect_example.fit_transform(text_example)
countvect_example.get_feature_names_out() # 교과서 코드와 함수명이 바뀌었습니다. 

* 쓰기 편하게 array로 바꿔봅시다

In [ ]:
countvect_example.transform(text_example).toarray()

In [ ]:
countvect_example.vocabulary_

### 베르누이 나이브 베이지안 분류기
* sklearn에 구현된 나이브 베이즈 분류기는 여러 개가 있습니다.
  * 이번에는 베르누이 나이브 베이즈 분류기를 생각해 봅시다. 
* 베르누이 분류기의 가정: 모든데이터가 boolean이다 
  * 그래서 정수값이 들어가면 어떤 임계값(threshold) 기준으로 T/F로 변홥합니다

In [ ]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB(alpha=1, binarize=0)
clf.fit(X_example, y_example)

* fit을 한 이후에는 다양한 값들을 제공해 줍니다
  * 예를 들어서 아래의 `class_log_prior_`는 각 클래스의 확률의 로그값을 보여줍니다. 여기는 sports/not sports겠죠?

In [ ]:
print(clf.class_log_prior_)
print(np.exp(clf.class_log_prior_))

### 다항 나이브 베이즈 분류기
* 다항 나이브 분류기는 베르누이 분류기와 다르게 각 feature들이 이산형이지만, 이진값이 아니라 다양한 값을 가질 수 있습니다.
* 이런 경우에는 아래의 식에서

$P(Y_c|X_1, \cdots, X_n)=P(Y_c)\prod_{i=1}^{n}\frac{P(X_i|Y_c)}{P(X_i)}=\frac{P(Y_c)\prod_{i=1}^{n}P(X_i|Y_c)}{\prod_{i=1}^{n}P(X_i)}$

$P(X_i|Y_c) = \frac{\sum{tf(x_i, d \in Y_c)} + \alpha}{\sum {N_d \in Y_c} + \alpha V}$ 형태로 가능도 계산을 바꿔주게 됩니다. 

* 여기에서 
  * $x_i$: feature vector에 존재하는 단어를 의미합니다. 각 단어는 index i를 할당합니다
  * $\sum{tf(x_i, d in Y_c}$: 각 단어 $x_i$ 가 클래스 $Y_c$에 속하는 모든 문서에 존재하는 개수. 
     * 예를 들어 game이라는 단어가 sports class 문서들에 몇 번이나 나오는지 등.
  * $\alpha$ : smoothing parameter -> 위에서 본 카운트가 0인 값으로 인해 발생하는 문제를 제거해주는 역할을 합니다.
     * 위를 보면 곱이 0인 경우가 나오죠
  * $\sum {N_d \in Y_c}$: $Y_c$에 해당하는 문서들에 존재하는 모든 단어의 합
  * $V$: 모든 단어의 수, 즉 feature의 수 (feature 차원 수) 입니다

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1)
clf.fit(X_example, y_example)

### 가우시안 나이브 베이즈 분류기
* 마지막으로 이산형 데이터가 아닌 경우에 가우스 분포를 가정하는 아래와 같은 가우시안 나이브 베이즈 분류기를 쓸 수도 있습니다
  * 가정이 강해 보이지만, CLT를 고려하면 사실 크게 틀린 말도 아니긴 합니다.
* 이 경우에는 가능도가 아래와 같이 바뀝니다 (평균에서 멀어질수록 잘 안 일어나는 일!)

$p(x_i|Y_c)=\frac{1}{\sqrt{2 \pi {\sigma_{Y_i}}^2}}exp(-\frac{(x_i-\mu Y_c)^2}{{2 \sigma_{Y_i}}^2})$

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_example.toarray(), y_example)

## 20 newsgroup으로 분류 연습하기

### 20 newsgroup dataset
* 20 뉴스그룹은 1990년대부터 2000년대까지 뉴스그룹 (뉴스를 공유하는 이메일을 받는 그룹)의 뉴스들과 주제를 가지고, 텍스트가 어떤 주제와 관련되었는지를 보여줍니다. 
* 역시나 scikit-learn에서 기본으로 주는 연습용 데이터입니다

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

### 데이터셋 불러오기
* scikit-learn 연습용 데이터이므로 sklearn.dataset에 포함되어 있습니다.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')
news.keys()

* data: 실제 데이터
* filenames: 저장된 데이터의 위치
* target_names: y 값의 이름
* target: y 값의 인덱스
* DESCR: 데이터 설명

In [ ]:
#print(news['DESCR']) #주의: 꽤 깁니다

* 데이터 하나를 봅시다. 데이터는 간단하게 이메일 형태로 되어있습니다.
* 상단은 메타, 하단은 메일 내용

In [ ]:
print(news.data[0])

* y값에 해당하는 target과 target_names를 봅시다. 
  * 뉴스그룹의 총 수는 20개 입니다(그래서 20 newsgroups...)

In [ ]:
news.target

In [ ]:
news.target_names

### 전처리. 전처리. 전처리.
* 먼저 데이터프레임을 만들어서 데이터를 정리해 봅시다. 

In [ ]:
news_df = pd.DataFrame({'News' : news.data, 'Target' : news.target})
news_df.head()

* Target의 정보를 실제 이름으로 바꿔봅시다. 

In [ ]:
target_dict = {idx:name for idx, name in enumerate(news.target_names)}
news_df["Target"] = news_df["Target"].replace(target_dict)

* 그리고 간단한 텍스트 전처리를 수행합시다

In [ ]:
def data_cleansing(df):
    # 정규표현식으로 이메일 제거 \w = [A-z0-9__]
    delete_email = re.sub(r'\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b', ' ', df) 
    # 이메일을 지운 후에 불필요한 숫자 제거
    delete_number = re.sub(r'\b|\d+|\b', ' ', delete_email) 
    # 알파벳이 아닌 문자를 모두 제거. \W = [^A-z0-9__]
    delete_non_word = re.sub(r'\b[\W]+\b', ' ', delete_number)
    # 공백 정규화 (띄어쓰기로 분할하고 다시 join해서 띄어쓰기를 "한 개만" 가지도록 합니다
    cleaning_result = ' '.join(delete_non_word.split())
    return cleaning_result

news_df.loc[:, 'News'] = news_df['News'].apply(data_cleansing)
news_df.head()

### 벡터화하기
* 먼저 단어의 수를 줄이고 비슷한 뜻을 가진 경우를 한 가지로 묶기 위해 어간(stem)추출을 해 봅시다.
* 흔히 쓰이는 nltk를 쓰겠습니다

In [ ]:
#!conda install -n ml2024 -c conda-forge nltk -y # if you do not have nltk already.
#!pip install nltk # if you prefer pip

* stem을 하면 어간이 같은 단어는 다 한가지의 기본형으로 묶이게 됩니다.
  * 시제나 의미등이 일부 손실되므로, 하는 것이 꼭 좋다고 말할 수는 없습니다만...

In [ ]:
from nltk import stem
stmmer = stem.SnowballStemmer("english")
sentence = 'looking looks looked'
[stmmer.stem(word) for word in sentence.split()]

In [ ]:
stmmer.stem("images"), stmmer.stem("imaging"), stmmer.stem("imagination")

* Vectorizer는 CountVectorizer 이외에도 여러가지가 있습니다.
* 한가지 예는 TF-IDF (Term Frequency - Inverse Document Frequency) Vectorizer 입니다.
  * 문서의 특징을 나타내기 위해서 두 가지조건을 사용합니다 
    * 특정 문서에 단어가 "많이 나오고" (Term frequency)
    * 그 단어가 다른 문서에 "잘 안 쓰이면" (Inverse Document Frequency)
    * 이 단어는 그 문서의 특징을 잘 나타낸다 
* IDF는 여러 수식이 있지만, 그 중 하나는 아래와 같은 로그 식을 쓰는 것입니다
  * $\log (\frac{N}{df_i})$
* tf-idf는 위 식을 쓰면 아래같이 쓰게 됩니다.
  * tf-idf $= tf_{i, j}\log (\frac{N}{df_i})$

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

enlish_stemmer = nltk.stem.SnowballStemmer("english")
class StemmedCountVectorizer(CountVectorizer): # CounterVectorizer를 상속받아 어간만 사용하는 Vectorizer를 만듭니다.
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: (enlish_stemmer.stem(w) for w in analyzer(doc))
    
from sklearn.feature_extraction.text import TfidfVectorizer

enlish_stemmer = nltk.stem.SnowballStemmer("english")
class StemmedTfidfVectorizer(TfidfVectorizer): # TfidfVectorizer를 상속받아 어간만 사용하는 Vectorizer를 만듭니다.
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc: (enlish_stemmer.stem(w) for w in analyzer(doc))

### 모델링하기
* 지금 우리는 여러가지의 vectorizer와 여러가지의 ML 알고리즘이 있습니다. 
* fit 과 transform을 이용한 파이프라인을 만들어 봅시다.

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

vectorizer = [CountVectorizer(), TfidfVectorizer(), StemmedCountVectorizer(), StemmedTfidfVectorizer()]
# algorithms = [BernoulliNB(), MultinomialNB(), GaussianNB(), LogisticRegression()]
algorithms = [MultinomialNB(), LogisticRegression(max_iter=500)]

pipelines = []

* `itertools.product` 를 이용해서 두 array의 조합의 경우의 수를 pipeline으로 만들어봅시다

In [ ]:
import itertools
for case in list(itertools.product(vectorizer, algorithms)):
    pipelines.append(make_pipeline(*case))
pipelines

* 이제 여러 알고리즘의 하이퍼파라미터를 만들어 봅시다.
  * 기본적으로는 클래스 이름__하이퍼파라미터의 이름 형태로 되어 있습니다
* 먼저 벡터화 파라미터를 고릅시다

In [ ]:
ngrams_params = [(1,1),(1,3)]
stopword_params = ["english"]
lowercase_params = [True, False]
max_df_params = np.linspace(0.4, 0.6, num=6)
min_df_params = np.linspace(0.0, 0.0, num=1)

attributes = {"ngram_range":ngrams_params, "max_df":max_df_params,"min_df":min_df_params,
              "lowercase":lowercase_params,"stop_words":stopword_params}
vectorizer_names = ["countvectorizer","tfidfvectorizer","stemmedcountvectorizer","stemmedtfidfvectorizer"]
vectorizer_params_dict = {}

for vect_name in vectorizer_names:
    vectorizer_params_dict[vect_name] = {}
    for key, value in attributes.items():
        param_name = vect_name + "__" + key
        vectorizer_params_dict[vect_name][param_name] = value

In [ ]:
# vectorizer_params_dict

* 그 다음은 각 알고리즘의 파라미터를 고릅시다

In [ ]:
algorithm_names = ["multinomialnb", "logisticregression"]

algorithm_params_dict = {}
alpha_params = np.linspace(1.0, 1.0, num=1)
for i in range(1):
    algorithm_params_dict[algorithm_names[i]] = {
        algorithm_names[i]+ "__alpha" : alpha_params
    }
c_params = [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]

algorithm_params_dict[algorithm_names[1]] = [{
    #"logisticregression__multi_class" : ["multinomial"],
    "logisticregression__solver" : ["saga"], #sag나 saga는 multinomial이 기본입니다.
    "logisticregression__penalty" : ["l1"],
    "logisticregression__C" : c_params
    },{ 
    #"logisticregression__multi_class" : ["ovr"],
    "logisticregression__solver" : ['liblinear'], #Sublinear solver를 쓰면 기본이 ovr입니다. (교과서 코드 warning 해결)
    "logisticregression__penalty" : ["l2"],
    "logisticregression__C" : c_params
    }
    ]    

In [ ]:
#algorithm_params_dict

* 마지막으로 파이프라인에 이 파라미터들을 차례대로 입력해 봅시다

In [ ]:
pipeline_params= []
for case in list(itertools.product(vectorizer_names, algorithm_names)):
    vect_params = vectorizer_params_dict[case[0]].copy()
    algo_params = algorithm_params_dict[case[1]]  
    
    if isinstance(algo_params, dict):
        vect_params.update(algo_params)
        pipeline_params.append(vect_params)
    else:
        temp = []
        for param in algo_params:
            vect_params.update(param)
            temp.append(vect_params)
        pipeline_params.append(temp)

In [ ]:
#pipeline_params

### 학습하기
* 이제 학습을 해 봅시다.
  * 이미 파이프라인을 다 만들었으니, 큰 문제는 없습니다.
* 먼저 feature(X)와 target(Y)를 아래와 같이 정리해봅시다.
  * `LabelEncoder`를 사용합니다 

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_data = news_df.loc[:, 'News'].tolist()
y_data = news_df['Target'].tolist()
y = LabelEncoder().fit_transform(y_data) # y_data 값을 기준으로 label으로 바꿔줍니다

In [ ]:
#y

* 여러 개의 모델과 파라미터를 바꿔가며 최적 모델을 찾는 방법으로 grid search 가 있습니다
  * 파라미터의 모든 조합을 테스트하며 가장 좋은 성능의 모델을 찾는 것입니다
  * `sklearn.model_selection.GridSearchCV` 로 구현되어 있습니다.

In [ ]:
pipeline_params

* n_jobs: 동시에 몇 개의 worker가 돌아갈지 (병렬화!). -1 means using all processors. 
* cv: cross-validation splitting strategy 를 정합니다. 기본 값은 5-fold cross validation입니다. 

![IMG](./W12/grid_search_cross_validation.png)